In [19]:
# ==========================================
# CrimeHotspotSim - Data Preprocessing Notebook
# ==========================================
# Goal:
# Explore, clean, and prepare crime and contextual datasets
# for spatio-temporal prediction and hotspot simulation.
# ==========================================

# --- 1. Import Libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import os

# Set visualization style
sns.set(style="whitegrid", palette="muted")

In [20]:
# --- 2. Load Dataset ---
# Path to raw dataset
crime_path = "../data/raw/Crime_Data_from_2020_to_Present.csv" # Name of the Dataset (You can name it whatever you want.)

# Load data
df = pd.read_csv(crime_path)

# Basic inspection
print("Dataset shape:", df.shape)
df.head()

Dataset shape: (1004991, 28)


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,211507896,04/11/2021 12:00:00 AM,11/07/2020 12:00:00 AM,845,15,N Hollywood,1502,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,7800 BEEMAN AV,NaN,34.2124,-118.4092
1,201516622,10/21/2020 12:00:00 AM,10/18/2020 12:00:00 AM,1845,15,N Hollywood,1521,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,IC,Invest Cont,230.0,NaN,NaN,NaN,ATOLL AV,N GAULT,34.1993,-118.4203
2,240913563,12/10/2024 12:00:00 AM,10/30/2020 12:00:00 AM,1240,9,Van Nuys,933,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,14600 SYLVAN ST,NaN,34.1847,-118.4509
3,210704711,12/24/2020 12:00:00 AM,12/24/2020 12:00:00 AM,1310,7,Wilshire,782,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND ...,...,IC,Invest Cont,331.0,NaN,NaN,NaN,6000 COMEY AV,NaN,34.0339,-118.3747
4,201418201,10/03/2020 12:00:00 AM,09/29/2020 12:00:00 AM,1830,14,Pacific,1454,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,IC,Invest Cont,420.0,NaN,NaN,NaN,4700 LA VILLA MARINA,NaN,33.9813,-118.4350


In [21]:
# --- 3. Initial Exploration ---
# View dataset info
df.info()

# Check missing values
df.isna().sum()

# Quick statistics
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004991 entries, 0 to 1004990
Data columns (total 28 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   DR_NO           1004991 non-null  int64  
 1   Date Rptd       1004991 non-null  object 
 2   DATE OCC        1004991 non-null  object 
 3   TIME OCC        1004991 non-null  int64  
 4   AREA            1004991 non-null  int64  
 5   AREA NAME       1004991 non-null  object 
 6   Rpt Dist No     1004991 non-null  int64  
 7   Part 1-2        1004991 non-null  int64  
 8   Crm Cd          1004991 non-null  int64  
 9   Crm Cd Desc     1004991 non-null  object 
 10  Mocodes         853372 non-null   object 
 11  Vict Age        1004991 non-null  int64  
 12  Vict Sex        860347 non-null   object 
 13  Vict Descent    860335 non-null   object 
 14  Premis Cd       1004975 non-null  float64
 15  Premis Desc     1004403 non-null  object 
 16  Weapon Used Cd  327247 non-null   fl

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
count,1.004991e+06,1004991,1004991,1.004991e+06,1.004991e+06,1004991,1.004991e+06,1.004991e+06,1.004991e+06,1004991,...,1004990,1004991,1.004980e+06,69160.000000,2314.000000,64.00000,1004991,154236,1.004991e+06,1.004991e+06
unique,NaN,1896,1879,NaN,NaN,21,NaN,NaN,NaN,140,...,6,6,NaN,NaN,NaN,NaN,66566,10413,NaN,NaN
top,NaN,02/02/2023 12:00:00 AM,01/01/2020 12:00:00 AM,NaN,NaN,Central,NaN,NaN,NaN,VEHICLE - STOLEN,...,IC,Invest Cont,NaN,NaN,NaN,NaN,800 N ALAMEDA ST,BROADWAY,NaN,NaN
freq,NaN,929,1164,NaN,NaN,69670,NaN,NaN,NaN,115190,...,802862,802862,NaN,NaN,NaN,NaN,2598,2486,NaN,NaN
mean,2.202215e+08,NaN,NaN,1.339900e+03,1.069174e+01,NaN,1.115633e+03,1.400348e+00,5.001568e+02,NaN,...,NaN,NaN,4.999174e+02,958.101258,984.015990,991.21875,NaN,NaN,3.399821e+01,-1.180909e+02
std,1.319718e+07,NaN,NaN,6.510613e+02,6.110255e+00,NaN,6.111605e+02,4.899691e-01,2.052731e+02,NaN,...,NaN,NaN,2.050736e+02,110.354348,52.350982,27.06985,NaN,NaN,1.610713e+00,5.582386e+00
min,8.170000e+02,NaN,NaN,1.000000e+00,1.000000e+00,NaN,1.010000e+02,1.000000e+00,1.100000e+02,NaN,...,NaN,NaN,1.100000e+02,210.000000,310.000000,821.00000,NaN,NaN,0.000000e+00,-1.186676e+02
25%,2.106169e+08,NaN,NaN,9.000000e+02,5.000000e+00,NaN,5.870000e+02,1.000000e+00,3.310000e+02,NaN,...,NaN,NaN,3.310000e+02,998.000000,998.000000,998.00000,NaN,NaN,3.401470e+01,-1.184305e+02
50%,2.209159e+08,NaN,NaN,1.420000e+03,1.100000e+01,NaN,1.139000e+03,1.000000e+00,4.420000e+02,NaN,...,NaN,NaN,4.420000e+02,998.000000,998.000000,998.00000,NaN,NaN,3.405890e+01,-1.183225e+02
75%,2.311103e+08,NaN,NaN,1.900000e+03,1.600000e+01,NaN,1.613000e+03,2.000000e+00,6.260000e+02,NaN,...,NaN,NaN,6.260000e+02,998.000000,998.000000,998.00000,NaN,NaN,3.416490e+01,-1.182739e+02


In [23]:
# --- 4. Handle Missing or Invalid Data ---
# Drop rows with missing coordinates
df = df.dropna(subset=['LAT', 'LON'])

# Drop duplicates if any
df = df.drop_duplicates()

# Example: Fill missing categorical data with mode
if 'Crime_Type' in df.columns:
    df['Crime_Type'].fillna(df['Crime_Type'].mode()[0], inplace=True)

# Confirm cleaning
df.isna().sum()

DR_NO                   0
Date Rptd               0
DATE OCC                0
TIME OCC                0
AREA                    0
AREA NAME               0
Rpt Dist No             0
Part 1-2                0
Crm Cd                  0
Crm Cd Desc             0
Mocodes            151619
Vict Age                0
Vict Sex           144644
Vict Descent       144656
Premis Cd              16
Premis Desc           588
Weapon Used Cd     677744
Weapon Desc        677744
Status                  1
Status Desc             0
Crm Cd 1               11
Crm Cd 2           935831
Crm Cd 3          1002677
Crm Cd 4          1004927
LOCATION                0
Cross Street       850755
LAT                     0
LON                     0
dtype: int64